In [8]:
!pip install symspellpy importlib_resources

import numpy as np
import sys
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
import symspellpy
from symspellpy import SymSpell
import importlib_resources as pkg_resources

Braille-Text Mappings (Unicode Braille characters)

In [9]:
braille_to_text = {
    '⠁': 'a', '⠃': 'b', '⠉': 'c', '⠙': 'd', '⠑': 'e',
    '⠋': 'f', '⠛': 'g', '⠓': 'h', '⠊': 'i', '⠚': 'j',
    '⠅': 'k', '⠇': 'l', '⠍': 'm', '⠝': 'n', '⠕': 'o',
    '⠏': 'p', '⠟': 'q', '⠗': 'r', '⠎': 's', '⠞': 't',
    '⠥': 'u', '⠧': 'v', '⠺': 'w', '⠭': 'x', '⠽': 'y',
    '⠵': 'z', ' ': ' ', '⠀': ' '  # Adding Braille space
}

text_to_braille = {v: k for k, v in braille_to_text.items()}

Initialize SymSpell for spell correction

In [10]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

Loading the dictionary from the correct path


In [15]:
try:
    dictionary_path = pkg_resources.files("symspellpy").joinpath("frequency_dictionary_en_82_765.txt")
    sym_spell.load_dictionary(str(dictionary_path), term_index=0, count_index=1)
except Exception as e:
    print(f"Error loading dictionary: {e}")

Auto-Correction using SymSpell

In [16]:
def auto_correct(text):
    if text:  # Check if text is not empty
        suggestions = sym_spell.lookup(text, symspellpy.Verbosity.CLOSEST, max_edit_distance=2)
        return suggestions[0].term if suggestions else text
    return text  # Return the original text if empty

Conversion Functions

In [17]:
def braille_to_text_conversion(braille_str):
    text = ''.join([braille_to_text.get(c, '?') for c in braille_str])
    return auto_correct(text)  # Apply AI-based auto-correction

def text_to_braille_conversion(text_str):
    return ''.join([text_to_braille.get(c.lower(), '?') for c in text_str])

CLI for Braille Converter

In [18]:
def cli_converter():
    print("\nBraille Converter CLI")
    print("=====================")
    while True:
        mode = input("Choose mode (1: Text to Braille, 2: Braille to Text, 0: Exit): ")
        if mode == "0":
            print("Exiting Braille Converter.")
            break
        text = input("Enter text or Braille: ")
        if mode == "1":
            print("Converted to Braille:", text_to_braille_conversion(text))
        elif mode == "2":
            print("Converted to Text:", braille_to_text_conversion(text))
        else:
            print("Invalid option! Please choose again.")


Run CLI Mode

In [19]:
if __name__ == "__main__":
    cli_converter()


Braille Converter CLI
Choose mode (1: Text to Braille, 2: Braille to Text, 0: Exit): 1
Enter text or Braille: harshit
Converted to Braille: ⠓⠁⠗⠎⠓⠊⠞
Choose mode (1: Text to Braille, 2: Braille to Text, 0: Exit): 2
Enter text or Braille: my "n is h>%it
Converted to Text: ?? ?? ?? ?????
Choose mode (1: Text to Braille, 2: Braille to Text, 0: Exit): 2
Enter text or Braille: 0
Converted to Text: a
Choose mode (1: Text to Braille, 2: Braille to Text, 0: Exit): 0
Exiting Braille Converter.
